In [ ]:
import os

dxf_folder = r"C:\Users\Rafael\Desktop\training_dataset\dxf files\9-characters\individual"
dxf_files = [os.path.join(dxf_folder, file) for file in os.listdir(dxf_folder) if file.endswith('.dxf')]

print(f"Found {len(dxf_files)} DXF files. Processing...")
print(f"➡️ No need for width variation for character metrics")

In [ ]:
from typing import Dict
import time
from tqdm import tqdm
import win32com.client
from shapely.geometry import box
from collections import defaultdict

# Start AutoCAD
acad = win32com.client.Dispatch("AutoCAD.Application")
acad.Visible = True # optional: run in background
time.sleep(5)  # Give time for AutoCAD to fully open

# Timing Settings
open_delay = 1.0
activate_delay = 0.5
close_delay = 0.2

characterMetrics: Dict[str, Dict[str, Dict]] = defaultdict(dict)
for file in tqdm(dxf_files, desc="Building characterMetrics"):
    
    doc = acad.Documents.Open(file); time.sleep(open_delay)
    doc.Activate(); time.sleep(activate_delay)
    modelSpace = doc.ModelSpace
    
    text = next(entity for entity in modelSpace)
    font = acad.ActiveDocument.TextStyles.Item(text.StyleName).FontFile.lower()
    character = text.TextString
    min_point, max_point = text.GetBoundingBox()
    bbox = box(min_point[0], min_point[1], max_point[0], max_point[1])
    width = max_point[0] - min_point[0]
    
    characterMetrics[font][character] = {"bbox": bbox, "width": width}
    
    doc.Close(False)
    time.sleep(close_delay)


Building characterMetrics: 100%|██████████| 238/238 [07:49<00:00,  1.97s/it]


In [3]:
import pickle

output_file = "character_metrics.pkl"  # or a full path if you want
with open(output_file, "wb") as f: pickle.dump(characterMetrics, f)
print(f"Saved character metrics to {output_file}")

Saved character metrics to character_metrics.pkl
